In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("lab-ml").getOrCreate()
spark

## Read in Scraped Data

In [2]:
df = spark.read.parquet("s3://aws-logs-780494741284-us-east-1/dataset/project_graph_data1")

In [4]:
df1 = df.select(['target'])
df2 = df.select(['source'])
df1 = df1.selectExpr("target as urls")
df2 = df2.selectExpr("source as urls")

df_c = df1.unionByName(df2)
df_c.printSchema()

root
 |-- urls: string (nullable = true)



## Create Graph for all the URLs

In [5]:
from operator import itemgetter

indexer = (df_c.select("urls").distinct()
    .rdd.map(itemgetter(0)).zipWithIndex()
    .toDF(["urls", "label"]))

df_c = df_c.join(indexer, ["urls"])

In [7]:

dic = dict(df_c.rdd.map(lambda x: (x['urls'], x['label'])).collect())
dic.get('http//0.ssbet138.cn/WWW.XPJ80886.COM/')


75

In [8]:
from pyspark.sql.functions import udf
def url_node(s):
    node = dic.get(s)
    return node

ur_udf = udf(url_node)

In [9]:
df_graph = df

## Create Edge for all URLs

In [10]:
df_graph_new = df_graph.withColumn("source", ur_udf('source'))

In [11]:
df_graph_new = df_graph_new.withColumn("target", ur_udf('target'))

In [12]:
df_graph_new.take(5)

[Row(source='14617', target='2486'),
 Row(source='3809', target='14553'),
 Row(source='3809', target='1820'),
 Row(source='3809', target='9445'),
 Row(source='3809', target='7173')]

In [13]:
df_graph_new.count()

31566

## Write Graph and Edge Datasets to S3

In [14]:
df_c.write.parquet("s3://aws-logs-780494741284-us-east-1/dataset/graph_node1",mode="overwrite")

In [17]:
df_graph_new.select('source').distinct().count()

51

In [15]:

df_graph_new.write.parquet("s3://aws-logs-780494741284-us-east-1/dataset/graph_edges1",mode="overwrite")